# Communications example
This notebook presents an example of a spread spectrum communications system that uses Gold Codes for transmitting data symbols.  A logic 0 is transmitted with $S_0$, and a logic 1 with $S_1$.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sps

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### GoldSequence Class Definition
The code below defines a Gold Code sequence generator.  The source for this code is by Christian Nelson at Lund University: https://github.com/chrinels/gold-sequence/blob/master/gold.py

In [ ]:
# This code is originally from https://github.com/chrinels/gold-sequence/blob/master/gold.py
# by Christian Nelson (chrinels).  Last accessed 16 Sep, 2019
class LFSR:

    def __init__(self, taps, init_register, output=None, samples_per_frame=None, matlab=False):
        if taps[-1] != 0:
            raise AttributeError("The first and last taps must be connected.")
        if taps[0] != len(init_register):
            raise AttributeError("The first and last taps must be connected.")
        if output is None:
            output = [len(init_register)]
        if any([((i > len(init_register)) | (i < 1)) for i in output]):
            raise AttributeError("The output tap(s) must exist!")

        if samples_per_frame is None:
            self.samples = 2**len(init_register) - 1
        else:
            self.samples = samples_per_frame

        self.init_register = init_register.copy()
        self.register = init_register.copy()
        self.output = output
        if matlab:
            self.taps = [taps[0] - tap for tap in reversed(taps)]
            self.taps = self.taps[:-1]
        else:
            self.taps = taps[:-1]
        self.cycled_through = False

    def reset(self):
        self.register = self.init_register.copy()

    def step(self):
        frame = []
        for _ in range(self.samples):
            frame.append(self.shift())
        return frame

    def shift(self):
        out = [self.register[i-1] for i in self.output]
        if len(out) > 1:
            out = sum(out) % 2
        else:
            out = out[0]

        feedback = sum([self.register[i-1] for i in self.taps]) % 2
        for i in reversed(range(len(self.register) - 1)):
            self.register[i+1] = self.register[i]

        self.register[0] = feedback
        if self.register == self.init_register:
            self.cycled_through = True
        return out


class GoldSequence:
    """
       The GoldSequence object generates a Gold sequence. 
       Gold sequences form a large class of sequences 
       that have good periodic cross-correlation properties.
    """

    def __init__(self, first_polynomial = [6, 1, 0],
                 first_initial_conditions = [ 0, 0, 0, 0, 0, 1],
                 second_polynomial = [6, 5, 2, 1, 0], 
                 second_initial_conditions = [ 0, 0, 0, 0, 0, 1],
                 samples_per_frame=None, index=0, matlab= True, debug=False):
        self.mls1 = LFSR(first_polynomial, first_initial_conditions, samples_per_frame=samples_per_frame, matlab=matlab)
        self.mls2 = LFSR(second_polynomial, second_initial_conditions, samples_per_frame=samples_per_frame, matlab=matlab)
        self.index = index
        self.samples_per_frame = samples_per_frame
        self.cycled_through = False
        self.debug = debug

    def step(self):
        u = self.mls1.step()
        v = np.roll(self.mls2.step(), -self.index).tolist()
        g = (np.logical_xor(u, v) * 1).tolist()
        if self.debug:
            print("u = {}\nv = {}\nG = {}".format(u, v, g))
        if self.mls1.cycled_through or self.mls2.cycled_through:
            self.cycled_through = True
        self.index = (self.index + 1) % self.samples_per_frame
        return g

    def reset(self):
        self.mls1.reset()
        self.mls2.reset()

### Generate sequences for symbols
Here we generate two sequences from the Gold Sequence generator.  These will be used to define the two symbols, $S_0$ and $S_1$.

In [ ]:
hgld = GoldSequence(samples_per_frame = 63)

x = hgld.step()
y = hgld.step()

### Now define S0 and S1 as signals that are +/- 1
The sequence generator produces binary sequences which we will convert to symbols with no d.c. value.

In [ ]:
s0 = 2*np.array(x) - 1
s1 = 2*np.array(y) - 1

### Define the possible received sequences
Here we consider two consecutive symbols that are received.  With two possible symbols, there are four possibilities for two consecutive symbols.

In [ ]:
s00 = np.concatenate((s0, s0))
s01 = np.concatenate((s0, s1))
s10 = np.concatenate((s1, s0))
s11 = np.concatenate((s1, s1))

### Define plotting functions

In [ ]:
def plot_stem(ss, s, ylabel, name):
    """
       Create a stem plot according to correlation between ss and s.
       INPUT:
            ss (array-like): Sequence for correlation.
            s  (array-like): Sequence for correlation.
            ylabel (string): The label for y-axis.
            name   (string): The name used to save plot.
    """
    plt.figure(figsize = (16, 8))
    
    plt.rcParams.update({'font.size': 16})
    correlation = np.correlate(ss, s, "full")/63
    (markerLines, stemLines, baseLines) = plt.stem(np.arange(0, 63),
                                                   correlation[62: 125],
                                                   use_line_collection=True)
    plt.setp(baseLines, color = 'black', linewidth=1) 
    markerLines.set_markerfacecolor('none')
    
    plt.xlim([-1, 63])
    plt.ylim([-1.1, 1.1])
    plt.xlabel('Delay')
    plt.ylabel(r'$\rho_{S_{%s}S_%s}(l)$'%(ylabel[0],ylabel[1]))
    plt.title('Correlation between $S_{%s}$ and $S_%s$'%(ylabel[0],ylabel[1]))
    
    if not is_jupyter():
        plt.savefig(name)

In [ ]:
def plot_amp(y, ylabel, title, name):
    """
       Plot the amplitude.
       INPUT:
           y  (array-like): The vertical coordinates of the data points.
           ylabel (string): The label for y-axis.
           title  (string): The title of figure.
           name   (string): The name used to save figure.
    """
    plt.figure(figsize = (16, 8))
    
    plt.rcParams.update({'font.size': 16})
    step = np.arange(0, len(y)/63, 1/63)
    plt.plot(step, y)
    plt.xlim([step[0],step[-1]])
    plt.xlabel('Time (symbols)')
    plt.ylabel(ylabel)
    plt.title(title)
    
    if not is_jupyter():
        plt.savefig(name)

In [ ]:
def plot_env(fs, num):
    """
       Plot the magnitude.
       INPUT:
           fs  (array-like): The output of digital filter.
           num        (int): Numver used to denote different signal.
    """
    plt.figure(figsize = (16, 8))
    
    plt.rcParams.update({'font.size': 16})
    plt.plot(np.arange(0, len(fs)/63, 1/63), abs(fs))
    plt.xlim([0,(len(fs)-1)/63])
    
    plt.xlabel('Time (symbols)')
    plt.ylabel('Magnitude')
    plt.title('Envelope of output of matched filter for $S_%s$'%num)

    if not is_jupyter():
        plt.savefig('Comms_example_envelope_S%s_output.pdf'%num)

### Now generate the plots of correlations

First determine the correlation between two consecutive 0 symbols ($S_0$) being transmitted with the matched filter for $S_0$.

In [ ]:
ylabel = ['00', '0']
name = 'Comms_example_rho_s0_s00.pdf'

plot_stem(s00, s0, ylabel, name)

and do the same for correlation with $S_1$.

In [ ]:
ylabel = ['00', '1']
name = 'Comms_example_rho_s1_s00.pdf'

plot_stem(s00, s1, ylabel, name)

Now examine the correlation between two consecutive 1 symbols being transmitted with the matched filter for $S_0$

In [ ]:
ylabel = ['11', '0']
name = 'Comms_example_rho_s0_s11.pdf'

plot_stem(s11, s0, ylabel, name)

and with the matched filter for $S_1$

In [ ]:
ylabel = ['11', '1']
name = 'Comms_example_rho_s1_s11.pdf'

plot_stem(s11, s1, ylabel, name)

There are four other cases to consider.  Firstly transmitting $S_0$ followed by $S_1$, correlated with $S_0$:

In [ ]:
ylabel = ['01', '0']
name = 'Comms_example_rho_s0_s01.pdf'

plot_stem(s01, s0, ylabel, name)

then correlated with $S_1$

In [ ]:
ylabel = ['01', '1']
name = 'Comms_example_rho_s1_s01.pdf'

plot_stem(s01, s1, ylabel, name)

Finally, consider transmitting $S_1$ followed by $S_0$, and correlating with $S_0$:

In [ ]:
ylabel = ['10', '0']
name = 'Comms_example_rho_s0_s10.pdf'

plot_stem(s10, s0, ylabel, name)

and with $S_1$.

In [ ]:
ylabel = ['10', '1']
name = 'Comms_example_rho_s1_s10.pdf'

plot_stem(s10, s1, ylabel, name)

The above plots show that where consecutive symbols are the same, then the correlations are high only when the pair of symbols are being correlated with the same symbol.  When the consecutive symbols are different, the correlation values at non-symbol delays are not as low as before, but there are still peaks when correlated with the matching symbol.

## Data communication example

We will now use matched filtering to detect symbols received over a transmission system.  We do not assume that the system is synchronised, so perform the correlation for all delays.

First generate the matched filters that we will use, and a data sequence that is to be transmitted.

In [ ]:
# matched filters 
hs0 = np.flip(s0, axis = 0)
hs1 = np.flip(s1, axis = 0)

# data sequence
sequence = np.concatenate((s0, s0, s1, s0, s1, s1, s0, s1,
                           s1, s1, s0, s0, s1, s0, s0))

### Additive noise
In the first instance we select a simple transmission medium where only Gaussian noise is added to the sequence.  The received signal is an attenuated version of the transmitted signal, plus noise.

In [ ]:
# The first channel will simply be an attenuation
# with additive white Gaussian noise

received = 0.7*sequence + np.random.randn(sequence.size)*0.4
ylabel = 'Amplitude'
title = 'Received signal'
name = 'Comms_example_simple_received.pdf'

plot_amp(received, ylabel, title, name)

The sequence is passed through the two matched filters - one detects the 0 symbols:

In [ ]:
# Now pass through the two filters
# use scipy.signal.lfilter function to filter data
# along one-dimension with an IIR or FIR filter.
fs0 = sps.lfilter(hs0, [1], received)


# And plot the results
ylabel = 'Filter output'
title = 'Output of matched filter for $S_0$'
name ='Comms_example_simple_S0_output.pdf'
plot_amp(fs0, ylabel, title, name)

and the other detects the 1 symbols:

In [ ]:
# Repeat for detecting symbols representing 1's
fs1 = sps.lfilter(hs1, [1], received)

ylabel = 'Filter output'
title = 'Output of matched filter for $S_1$'
name = 'Comms_example_simple_S1_output.pdf'
plot_amp(fs1, ylabel, title, name)

From the above plots, it is evident that the matched filters are effective in detecting the corresponding symbols in the noisy received sequence.

### Multipath channel
A more realistic communication medium will result in repeated versions of the transmitted signal being received, with different delays and attenuations.  Here we simulate a three path channel, with the addition of gaussian noise.  The same processing is employed to detect the transmitted sequence.

In [ ]:
# A more realistic channel will have signals arriving at different delays,
# with different attenuations, as well as additive noise
received = (0.7*sequence + 
            0.4*np.concatenate((np.zeros(14),sequence[0:sequence.size-14])) + 
            0.2*np.concatenate((np.zeros(19),sequence[0:sequence.size-19])) + 
            np.random.randn(sequence.size)*0.4)


# Now pass this new signal through the two filters
fs0 = sps.lfilter(hs0, [1], received)

# And plot the results
ylabel = 'Filter output'
title = 'Output of matched filter for $S_0$'
name = 'Comms_example_multipath_S0_output.pdf'
plot_amp(fs0, ylabel, title, name)

In [ ]:
fs1 = sps.lfilter(hs1, [1], received)
ylabel = 'Filter output'
title = 'Output of matched filter for $S_1$'
name = 'Comms_example_multipath_S1_output.pdf'
plot_amp(fs1, ylabel, title, name)

### Complex multipath channel
Practical communications also experience phase alteration of the received symbols.  Here we modify the three path channel to include a complex phase term on each multipath.  The resulting correlation will be complex, so we perform the detection on the magnitude only, and ignore the phase component.

In [ ]:
# Finally, a channel complex attenuation

received = (0.7*np.exp(1j*np.pi/3)*sequence + 
            0.4*np.exp(-1j*3*np.pi/5)*np.concatenate((np.zeros(14), sequence[0:sequence.size-14])) + 
            0.2*np.exp(-1j*2*np.pi/11)*np.concatenate((np.zeros(19), sequence[0:sequence.size-19])) + 
            np.random.randn(sequence.size)*0.4)

# Now pass this new signal through the two filters

fs0 = sps.lfilter(hs0, [1], received)
fs1 = sps.lfilter(hs1, [1], received)

# And plot the results
plot_env(fs0, '0')

In [ ]:
plot_env(fs1, '1')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.